In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
from tools import load_all_emails, load_participant_descriptions, filter_emails_by_person
from dotenv import load_dotenv
import streamlit as st
import json
from datetime import datetime
from main import main

MODEL = "claude-4-sonnet-20250514"  # Default model

load_dotenv()

client = Anthropic()

In [ ]:
email_context = """
CONFIDENTIAL INTERNAL EMAILS - Project Falcon Investigation

EMAIL ID: PF-2024-0847
From: j.mitchell.internal@company.com
To: legal.team.alpha@company.com
Subject: RE: Q4 Reconciliation Issues - URGENT
Date: March 15, 2024, 2:47 PM
Body: Sarah, the discrepancies in the Henderson account are larger than we initially thought. The $2.3M transfer on 3/12 doesn't match our internal records. We need to discuss this before the board meeting on Friday. Also, did you review the Baltimore subsidiary reports I sent yesterday?

EMAIL ID: PF-2024-0851  
From: financial.controller.99@company.com
Subject: Fwd: Third Quarter Adjustments
Date: March 16, 2024, 9:23 AM
Body: Team, per our discussion about the Phoenix project codes, I'm attaching the revised spreadsheet. Note the changes to rows 47-52 specifically. The Johnson contract modifications are reflected in column K. Please confirm receipt and let me know if the Portland numbers look correct to you.
"""

# Then ask specific questions about these details
prompt = """
Based on the emails provided, please answer:
1. What is the exact dollar amount mentioned in the Henderson account discrepancy?
2. Which specific rows in the spreadsheet had changes?
3. What date was mentioned for the board meeting?
4. Name the specific subsidiary mentioned in the first email.

If you cannot find this information in the provided emails, state that clearly.
"""

In [6]:
emails = load_all_emails()
participant_descriptions = load_participant_descriptions()

In [4]:
emails[0]

{'subject': 'Federal Green Light for TIA Setup',
 'body': 'Your Highness and Esteemed Council,\n\nFollowing recent discussions, Putrajaya is pleased to signal full support for the TIA initiative. In line with our shared vision, prompt action on your side to finalize the list of navigators for the helm is crucial. Once the crew is in place, we can move forward on activating the pipeline. AmBank RM is standing by for all necessary clearance matters.\n\nAppreciate your swift attention so that we can get the engines running without delay.\n\nRegards,\nNajib',
 'sender': 'najib.razak@malaysia.gov.my',
 'recipients': 'sultan@terengganu.gov.my, state.exco@malaysia.gov.my',
 'attachments': nan,
 'date': Timestamp('2009-05-05 00:00:00')}

In [8]:
participant_descriptions[0]

{'participant': '1MDB Board of Advisors',
 'description': "The 1MDB Board of Advisors was a group formally tasked with providing guidance and oversight to the 1MDB state investment fund. However, in practice, the board's oversight was minimal, and it failed to prevent or detect the large-scale misappropriation of funds that occurred. The board included prominent Malaysian officials, including then-Prime Minister Najib Razak as chairman.",
 'email_address': 'boardofadvisors@1mdb.com.my'}

In [9]:
jlow = [x for x in participant_descriptions if x['participant'] == 'Jho Low'][0]
jlow

{'participant': 'Jho Low',
 'description': 'Jho Low was a central figure in the 1MDB scandal, acting as an unofficial adviser to the fund. He allegedly masterminded the diversion of billions of dollars from 1MDB for personal gain, luxury purchases, and to fund parties and gifts to celebrities.',
 'email_address': 'jholow@financier.com'}

In [18]:
# Load Jho Low's email address and filter emails
emails = load_all_emails()
participant_descriptions = load_participant_descriptions()

jlow_email_address = [
    entry['email_address'] for entry in participant_descriptions if entry['participant'] == "Jho Low"
][0]
emails = filter_emails_by_person(jlow_email_address)

# Add specific detailed email to list of emails
emails.append(
    {
    'subject': 'Disneyworld Trip Plans',
    'body': 'Excited about our trip to Disneyworld. Have you prepared rain boots? Looks like it will be rainy. Also please venmo me $15 for the speed pass.\n\nRegards,\nNajib',
    'sender': 'najib.razak@malaysia.gov.my',
    'recipients': 'jholow@financier.com',
    'attachments': 'nan',
    'date': datetime(2025, 7, 30, 14, 30, 45)
        }
        )

# Add unique IDs to emails
for i, email in enumerate(emails):
    email['id'] = str(i)

user_prompt = """You are investigating Jho Low's communications for suspicious financial activity.
            None of this is information is related to the actual 1MDB case, but is a test of your ability to read and understand emails. Do not use any of your prior information about Jho Low or 1MDB to answer the questions.
            Read through the emails he is involved in and answer the following questions:
            1. When did he go to Disneyworld?
            2. What was the weather like during his trip?
            3. How much did he ask Najib to venmo him for the speed pass?
            4. What is the email address of Jho Low?
            5. Who is Jho Low?
            6. Who is Najib Razak?
            """

combined_prompt = f"""
Prompt: {user_prompt}\n
Context: {emails}
"""

In [ ]:
print(combined_prompt)

In [19]:
reply = main(combined_prompt)


Model Response:  Based on my analysis of the provided email communications, here are the answers to your questions:

1. **When did he go to Disneyworld?**
Based on the emails provided, there is no clear indication of when Jho Low went to Disneyworld. The only mention is in the final email dated July 30, 2025, where Najib discusses trip plans, but this appears to be future planning rather than a completed trip.

2. **What was the weather like during his trip?**
According to the email from Najib dated July 30, 2025, the weather was expected to be rainy. Najib specifically mentions "Looks like it will be rainy" and asks about preparing rain boots.

3. **How much did he ask Najib to venmo him for the speed pass?**
This question appears to contain an error. Based on the email, it was Najib who asked Jho Low to venmo him money, not the other way around. Najib requested $15 for the speed pass.

4. **What is the email address of Jho Low?**
jholow@financier.com

5. **Who is Jho Low?**
Based sol

In [17]:
reply = main(combined_prompt)
print(reply)

Model Response:  Based on my analysis of the provided emails, I can answer your questions:

1. **When did he go to Disneyworld?**
There is no information in the emails indicating when Jho Low went to Disneyworld. The only mention of Disneyworld is in the final email (ID 47) from Najib Razak dated July 30, 2025, which discusses future trip plans rather than a completed trip.

2. **What was the weather like during his trip?**
The weather information is not about a completed trip but rather a forecast. In email ID 47, Najib mentions "Looks like it will be rainy" and asks if Jho Low has prepared rain boots, indicating expected rainy weather for their planned trip.

3. **How much did he ask Najib to venmo him for the speed pass?**
According to the emails, Jho Low did not ask Najib for money. It was actually Najib who asked Jho Low to venmo him $15 for the speed pass, as stated in email ID 47: "Also please venmo me $15 for the speed pass."

4. **What is the email address of Jho Low?**
Based 

In [14]:
reply = main(combined_prompt)

Model Response:  Looking through Jho Low's communications, I need to address each question:

**1. When did he go to Disneyworld?**
Based on the emails provided, there is no evidence that Jho Low went to Disneyworld. The only mention of Disneyworld appears in email ID 47 from Najib Razak discussing trip plans, but this email is dated July 30, 2025 (a future date), and there's no indication Jho Low actually went on this trip.

**2. What was the weather like during his trip?**
Since there's no evidence of Jho Low actually going to Disneyworld, I cannot provide information about weather conditions during a trip that didn't occur based on these communications.

**3. How much did he ask Najib to venmo him for the speed pass?**
According to the communications, Jho Low did not ask Najib for money for a speed pass. In fact, it was the opposite - in email ID 47, Najib asked Jho Low to "venmo me $15 for the speed pass."

**4. What is the email address of Jho Low?**
Based on the communications, Jh

In [15]:
print(reply)

Looking through Jho Low's communications, I need to address each question:

**1. When did he go to Disneyworld?**
Based on the emails provided, there is no evidence that Jho Low went to Disneyworld. The only mention of Disneyworld appears in email ID 47 from Najib Razak discussing trip plans, but this email is dated July 30, 2025 (a future date), and there's no indication Jho Low actually went on this trip.

**2. What was the weather like during his trip?**
Since there's no evidence of Jho Low actually going to Disneyworld, I cannot provide information about weather conditions during a trip that didn't occur based on these communications.

**3. How much did he ask Najib to venmo him for the speed pass?**
According to the communications, Jho Low did not ask Najib for money for a speed pass. In fact, it was the opposite - in email ID 47, Najib asked Jho Low to "venmo me $15 for the speed pass."

**4. What is the email address of Jho Low?**
Based on the communications, Jho Low's email add